# notes:
* removed some old, redudant analysis code; it remains in coal-map-2020 notebook
* updated code to calculate TWh and CO2 estimates per plant, using Ember data
* updated code to show "EU27+UK" instead of "EU28"
* analysis code hopefully more clearly laid out to quickly furnish data for article and charts

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import pyproj
import re
import os, math
os.chdir('C:\\Users\\chiga\\Projects\\carbon_brief\\map_coal_plant_update\\data')
%matplotlib inline

# Import the data and tidy up

## import data

In [2]:
####### ENTER THE NAME OF THE SPREADSHEET HERE #########
name = 'Global-Coal-Plant-Tracker-July-2022'

# load the data from the "Units" tab of the spreadsheet
df = pd.read_excel(name + '.xlsx', sheet_name='Units')

In [3]:
####################################################################
# need to update the year flag below to the current year
####################################################################

current_year = 2022  # this is the year the data was published
current_year

2022

In [4]:
# rename some columns to match last year's data

df.rename(columns={'Year':'Start Year','RETIRED':'Retire Year'}, inplace=True)

# change status entries from "Pre-Permit" to "Pre-permit" to match last year
# make sure other status capitalised

df.replace('pre-permit','Pre-permit', inplace=True)
df.replace('mothballed','Mothballed', inplace=True)
df['Status'].replace('operating','Operating', inplace=True)
df['Status'].replace('construction','Construction', inplace=True)
df['Status'].replace('shelved','Shelved', inplace=True)
df['Status'].replace('announced','Announced', inplace=True)
df['Status'].replace('cancelled','Cancelled', inplace=True)
df['Status'].replace('retired','Retired', inplace=True)
df['Status'].replace('permitted','Permitted', inplace=True)


In [5]:
df["Status"].unique()

array(['Cancelled', 'Shelved', 'Announced', 'Operating', 'Pre-permit',
       'Permitted', 'Construction', 'Mothballed', 'Retired'], dtype=object)

In [6]:
df.Status.unique()

array(['Cancelled', 'Shelved', 'Announced', 'Operating', 'Pre-permit',
       'Permitted', 'Construction', 'Mothballed', 'Retired'], dtype=object)

## sort out start years

In [7]:
# find units with start years that are decades not years

values = [(df['Capacity (MW)'].loc[(df['Start Year']=='1960\'s')|(df['Start Year']=='1970\'s')|(df['Start Year']=='1980\'s')|(df['Start Year']=='1990\'s')].sum()),'MW with start year of 60s/70s/80s/90s']
print(*values) 
values = [(df['Capacity (MW)'].loc[(df['Start Year']=='1960\'s')|(df['Start Year']=='1970\'s')|(df['Start Year']=='1980\'s')|(df['Start Year']=='1990\'s')].count()), ' units with start year of 60s/70s/80s/90s']
print(*values) 

# database has been updated so these start years are no longer used

1070.0 MW with start year of 60s/70s/80s/90s
21  units with start year of 60s/70s/80s/90s


In [8]:
# assign start years for retired plants marked by decade

df['Start Year'].replace('1960\'s', '1965', inplace=True)
df['Start Year'].replace('1960s', '1965', inplace=True)
df['Start Year'].replace('1970\'s', '1975', inplace=True)
df['Start Year'].replace('1970s', '1975', inplace=True)
df['Start Year'].replace('1980\'s', '1985', inplace=True)
df['Start Year'].replace('1980s', '1985', inplace=True)
df['Start Year'].replace('1990\'s', '1995', inplace=True)
df['Start Year'].replace('1990s', '1995', inplace=True)

In [9]:
# convert Start Years and Capacity (MW) to numeric

df['Start Year'] = df[['Start Year']].apply(
    pd.to_numeric, errors='coerce')
df['Capacity (MW)'] = df[['Capacity (MW)']].apply(
    pd.to_numeric, errors='coerce')


## write file with missing data

In [10]:
# write info on missing data to a text file
# create the file

#date = name.replace('Global Coal Plant Tracker ','')

# make sure to update the filename, here Apr 2021

# write to the file
with open('Apr 2021' + '_missing_values.txt', 'w') as f:
    f.write('\n' + str(df['Unit'].loc[df['Capacity (MW)'].isnull()].count()) +
            ' units with no capacity data')
    f.write('\n\n' + str(df['Unit'].loc[df['Latitude'].isnull()].count()) +
            ' units (' + str(df['Capacity (MW)'].loc[df['Latitude'].isnull()]
                             .sum()) + ' MW) with no location data')
    f.write('\n\n' + str(df['Unit'].loc[(df['Latitude'].isnull()) & (
        df['Status'] == 'Operating')].count()) + ' operating units (' +
            str(df['Capacity (MW)'].loc[(df['Latitude'].isnull()) & (
                df['Status'] == 'Operating')].sum()) + ' MW) with no location')
    f.write('\n' + str(df['Unit'].loc[(df['Latitude'].isnull()) & (
        df['Status'] == 'Retired')].count()) + ' retired units (' +
            str(df['Capacity (MW)'].loc[(df['Latitude'].isnull()) & (
                df['Status'] == 'Retired')].sum()) + ' MW) with no location')
    f.write('\n' + str(df['Unit'].loc[(df['Latitude'].isnull()) & (
        df['Status'] == 'Announced')].count()) + ' announced units (' +
            str(df['Capacity (MW)'].loc[(df['Latitude'].isnull()) & (
                df['Status'] == 'Announced')].sum()) + ' MW) with no location')
    f.write('\n' + str(df['Unit'].loc[(df['Latitude'].isnull()) & (
        df['Status'] == 'Shelved')].count()) + ' shelved units (' +
            str(df['Capacity (MW)'].loc[(df['Latitude'].isnull()) & (
                df['Status'] == 'Shelved')].sum()) + ' MW) with no location')
    f.write('\n' + str(df['Unit'].loc[(df['Latitude'].isnull()) & (
        df['Status'] == 'Cancelled')].count()) + ' cancelled units (' +
            str(df['Capacity (MW)'].loc[(df['Latitude'].isnull()) & (
                df['Status'] == 'Cancelled')].sum()) + ' MW) with no location')

    f.write('\n\n' + str(df['Unit'].loc[df['Start Year'].isnull()].count()) +
            ' units (' + str(df['Capacity (MW)'].loc[df['Start Year'].isnull()]
                             .sum()) + ' MW) with no Start Year')
    f.write('\n' + str(df['Unit'].loc[(df['Start Year'].isnull(
    )) & (df['Status'] == 'Operating')].count()) + ' operating units (' + str(
        df['Capacity (MW)'].loc[(df['Start Year'].isnull()) & (
            df['Status'] == 'Operating')].sum()) + ' MW) with no Start Year')
    f.write('\n' + str(df['Unit'].loc[(df['Start Year'].isnull()) & (
        df['Status'] == 'Retired')].count()) + ' retired units (' +
            str(df['Capacity (MW)'].loc[(df['Start Year'].isnull()) & (
                df['Status'] == 'Retired')].sum()) + ' MW) with no Start Year')

    f.write('\n\n' + str(df['Unit'].loc[df['Retire Year'].isnull()].count()) +
            ' units (' + str(df['Capacity (MW)'].loc[df['Start Year'].isnull()]
                             .sum()) + ' MW) with no Retire Year')
    f.write('\n' + str(df['Unit'].loc[(df['Retire Year'].isnull(
    )) & (df['Status'] == 'Operating')].count()) + ' operating units (' + str(
        df['Capacity (MW)'].loc[(df['Retire Year'].isnull()) & (
            df['Status'] == 'Operating')].sum()) + ' MW) with no Retire Year')
    f.write('\n' + str(df['Unit'].loc[(df['Retire Year'].isnull(
    )) & (df['Status'] == 'Retired')].count()) + ' retired units (' + str(
        df['Capacity (MW)'].loc[(df['Retire Year'].isnull()) & (
            df['Status'] == 'Retired')].sum()) + ' MW) with no Retire Year')
    f.write('\n' + str(df['Unit'].loc[(df['Retire Year'].isnull()) & (
        df['Status'] == 'Retired') & (df['Country'] == 'China')].count(
        )) + ' retired units in China (' + str(
            df['Capacity (MW)'].loc[(df['Retire Year'].isnull()) &
                                    (df['Status'] == 'Retired') &
                                    (df['Country'] == 'China')].sum()) +
            ' MW) with no Retire Year')

## drop lines with missing capacity

In [11]:
# display basic info on plants with no capacity data

df[['Country','Plant','Status']][df['Capacity (MW)'].isna()]

,Country,Plant,Status


In [12]:
# remove these lines from the data

df = df[df['Capacity (MW)'].notnull()]

## import phaseout plans

In [13]:
# make sure country phaseout plans are reflected in the map
# list correct as at Mar 2020
# does not include subnational, city or company commitments
# eg South Chungcheong in South Korea, which will cut lifetime of plants to 25 yrs instead of 30

# import phaseout dates

phaseouts = pd.read_excel('phaseouts.xlsx')



#phaseouts.head()


# merge into main df, matching countries with phaseout dates

df = pd.merge(df,phaseouts,left_on=['Country'],right_on=['Country'], how='left')

df['Planned Retire'].fillna(df['Year'],inplace=True)

## delete missing lat/longs and tidy up coal types

In [14]:
# delete the rows without capacity or lat/long
df = df[df['Capacity (MW)'].notnull() & df['Latitude'].notnull()]

# calculate the CO2 factor (heat rate x emission factor x capacity factor x constant)
#df['facCO2'] = df['Heat rate'] * df['Emission factor'] * df[
#    'Capacity factor'] * 3.97347E-09

# tidy up the coal types
df['Coal type'].replace('ANTH', 'Anthracite', inplace=True)
df['Coal type'].replace('HARD', 'Hard coal', inplace=True)
df['Coal type'].replace('unknown', 'Unknown', inplace=True)
df['Coal type'].replace('lignite', 'Lignite', inplace=True)
df['Coal type'].replace('Sub-bit', 'Sub-bituminous', inplace=True)
df['Coal type'].replace('WSTBituminous', 'Bituminous', inplace=True)
df['Coal type'].replace('Bituminous/WSTBituminous', 'Bituminous', inplace=True)
df['Coal type'].replace('WSTCOAL', 'Bituminous', inplace=True)
df['Coal type'].replace('unknown(CCS90)', 'Unknown', inplace=True)
df['Coal type'].replace('Unknown(CCS90)', 'Unknown', inplace=True)
df['Coal type'].replace('Bituminous(CCS90)', 'Bituminous', inplace=True)
df['Coal type'].replace('Bituminous(CCS30)', 'Bituminous', inplace=True)
df['Coal type'].replace('Sub-bit(CCS90)', 'Sub-bituminous', inplace=True)
df['Coal type'].replace('Lignite(CCS90)', 'Lignite', inplace=True)
df['Coal type'].replace('ANTH/Bituminous', 'Anthracite/Bituminous', inplace=True)
df['Coal type'].replace('ANTH/Culm', 'Anthracite', inplace=True)


# make sure status labels are correct

df['Status'].replace('Pre','Pre-permit',inplace=True)

# tidy up the combustion types
df['Combustion technology'].replace('unknown', 'Unknown', inplace=True)

# strip plant names to remove bracketed text and -1, -A, -10, ' A ', etc etc

#df['Plant'] = df['Plant'].str.replace(r'\(.*\)','')
#df['Plant'] = df['Plant'].str.replace(r' [\w] ',' ')
#df['Plant'] = df['Plant'].str.replace('-[\w][\w]','')
#df['Plant'] = df['Plant'].str.replace('-[\w]','')
#df.loc[df['Plant'].str.contains('Morupule')]

# suppress errors in the "for" loops (I didn't understand them and they didn't seem to matter)
pd.options.mode.chained_assignment = None  # default='warn'

In [15]:
df['Coal type'].unique()

array(['Unknown', 'bituminous', 'anth', 'lignite/sub-bit', 'Lignite',
       'wstbituminous', 'sub-bit', 'wstcoal', 'bituminous/sub-bit',
       'unknown(ccs90)', 'lignite(ccs90)', 'sub-bit(ccs90)',
       'anth/bituminous', 'hard', 'lignite/bituminous',
       'bituminous(ccs90)', 'bituminous(ccs30)', 'anth/culm',
       'bituminous/wstbituminous'], dtype=object)

## fix issue with plant that starts and ends with exactly the same MW
* (Boryeong in South Korea starts at 3GW, adds 1GW then closes 1GW)

In [16]:
df['Capacity (MW)'].loc[df['Unit']=='Boryeong power station Unit 8'] = df['Capacity (MW)'] + 0.05

In [17]:
# fix coords for Anadyrskaya power station

df['Longitude'] = np.where(df.Plant == 'Anadyrskaya power station',177.496925,df.Longitude)

## replace missing retire years

In [18]:
df.columns

Index([                         'Tracker ID',
                                'TrackerLOC',
                                  'ParentID',
                                 'Wiki page',
                                   'Country',
        'Subnational unit (province, state)',
                                      'Unit',
                                     'Plant',
                              'Chinese Name',
                               'Other names',
                                     'Owner',
                                    'Parent',
                             'Capacity (MW)',
                                    'Status',
                                'Start Year',
                               'Retire Year',
                            'Planned Retire',
                     'Combustion technology',
                                 'Coal type',
                               'Coal source',
                                  'Location',
                'Local area (taluk

In [19]:
# correct the data type of retire year and start year 
df = df.replace(r'^\s*$', np.nan, regex=True)
df.loc[:, "Retire Year"] = df.loc[:, "Retire Year"].astype("float64")
df.loc[:, "Start Year"] = df.loc[:, "Start Year"].astype("float64")

In [20]:
###### code to replace missing retire years with random data weighted to known data ######

# get MW-weighted distribution of China retirements
df.loc[:, "Retire Year"] = df.loc[:, "Retire Year"]
china_retire1 = df[[
    'Retire Year', 'Capacity (MW)'
]].loc[(df['Status'] == 'Retired') & (df['Country'] == 'China') &
       (df['Retire Year'] >= 2000)].groupby('Retire Year').sum()

# convert to percent

china_retire = china_retire1.iloc[:, 0:].apply(
    lambda x: x / float(x.sum()), axis=0).reset_index()

# get MW-weighted distribution of non-China retirements

retire1 = df[['Retire Year', 'Capacity (MW)'
              ]].loc[(df['Status'] == 'Retired') & (df['Country'] != 'China') &
                     (df['Retire Year'] >= 2000)].groupby('Retire Year').sum()

# convert to percent

other_retire = retire1.iloc[:, 0:].apply(
    lambda x: x / float(x.sum()), axis=0).reset_index()

# create a random array weighted by China retirements
# make the array as long as the df
# append the random array to the df as a new column

from numpy.random import choice
list_of_candidates = china_retire['Retire Year'].tolist()
probability_distribution = china_retire['Capacity (MW)'].tolist()
number_of_items_to_pick = len(df)
draw = choice(
    list_of_candidates, number_of_items_to_pick, p=probability_distribution)
df['china-retire'] = draw

df['china-retire'] = np.where(df['Status'] == 'Retired', df['china-retire'],
                              2100)

# create a random array weighted by non-China retirements
# make the array as long as the df
# append the random array to the df as a new column

from numpy.random import choice
list_of_candidates = other_retire['Retire Year'].tolist()
probability_distribution = other_retire['Capacity (MW)'].tolist()
number_of_items_to_pick = len(df)
draw1 = choice(
    list_of_candidates, number_of_items_to_pick, p=probability_distribution)
df['other-retire'] = draw1

df['other-retire'] = np.where(df['Status'] == 'Retired', df['other-retire'],
                              2100)

# create dummy retirement years depending if the country is China or not

df['dummy-retire'] = np.where(df['Country'] == 'China', df['china-retire'],
                              df['other-retire'])

#df.loc[df['Plant'].str.contains('Belchatow')].iloc[:,4:20]

In [21]:
# check data types in case of "No numeric types to aggregate" error on Groupby
# df.dtypes

In [22]:
# convert data to numeric

df['Latitude'] = df['Latitude'].apply(pd.to_numeric, errors='coerce')
df['Planned Retire'] = df['Planned Retire'].apply(pd.to_numeric, errors='coerce')
df['Tracker ID'] = df['Tracker ID'].apply(pd.to_numeric, errors='coerce')



In [23]:
df.columns

Index([                         'Tracker ID',
                                'TrackerLOC',
                                  'ParentID',
                                 'Wiki page',
                                   'Country',
        'Subnational unit (province, state)',
                                      'Unit',
                                     'Plant',
                              'Chinese Name',
                               'Other names',
                                     'Owner',
                                    'Parent',
                             'Capacity (MW)',
                                    'Status',
                                'Start Year',
                               'Retire Year',
                            'Planned Retire',
                     'Combustion technology',
                                 'Coal type',
                               'Coal source',
                                  'Location',
                'Local area (taluk

In [24]:
# add columns for capacity-weighted heat rates and emissions factors

df['heatmw'] = df['Capacity (MW)']*df['Heat rate (Btu per kWh)']
df['emissionmw'] = df['Capacity (MW)']*df['Emission factor (kg of CO2 per TJ)']

# work on operating and retired plants

### select operating and retired plants

In [25]:
# select only those units that have been built ie operating now or already retired
open_closed = df.loc[(df['Status'] == 'Operating')
                     | (df['Status'] == 'Retired')
                     | (df['Status'] == 'Mothballed')]

# replace NaN Start Year with 1900, for operating and retired plants
open_closed['Start Year'].fillna(1900, inplace=True)

# some retire years have text; force to numbers
open_closed['Retire Year'] = open_closed[['Retire Year']].apply(
    pd.to_numeric, errors='coerce')

# make sure all rows have a retire year
# use dummy retire years for blank entries

open_closed['Retire Year'].fillna(open_closed['dummy-retire'], inplace=True)

# select only the columns we care about
open_closed = open_closed[[
    'Unit', 'Status', 'Plant', 'Country', 'Region', 'Start Year',
    'Retire Year', 'Planned Retire', 'Capacity (MW)', 'Combustion technology',
    'Coal type', 'Latitude', 'Longitude', 'Tracker ID',
    'Annual CO2 (million tonnes / annum)','heatmw','emissionmw'
]]
#open_closed.head()
#open_closed.loc[open_closed['Plant'].str.contains('Zouping')]

### create a listing for each unit in each year it's operating

In [26]:
# create a dataframe listing the capacity for each plant, in each year it is operating
byplant = pd.DataFrame()
byunit = pd.DataFrame()
for year in range(2000, current_year):
    operating = open_closed.loc[(open_closed['Start Year'] <= year)
                                & (open_closed['Retire Year'] > year)]
    #    operating['Year'] = year
    #    operating['Age'] = operating['Year'] - operating['Start Year']
    #    operating['Start Year'] = operating['Start Year'].astype('int' , inplace=True)


    #operating_plants['Year'] = year
    operating['Year'] = year
    #byplant = byplant.append(operating_plants)
    byunit = byunit.append(operating)



In [27]:
# double check we have numeric dtypes in columns we need for the groupby

byunit[['Plant', 'Capacity (MW)', 'Latitude', 'Tracker ID', 'Combustion technology',
    'Start Year', 'Retire Year', 'Annual CO2 (million tonnes / annum)', 'Year',
    'Planned Retire','heatmw','emissionmw']].dtypes

Plant                                   object
Capacity (MW)                          float64
Latitude                               float64
Tracker ID                             float64
Combustion technology                   object
Start Year                             float64
Retire Year                            float64
Annual CO2 (million tonnes / annum)    float64
Year                                     int64
Planned Retire                         float64
heatmw                                 float64
emissionmw                             float64
dtype: object

### group the units by plant name

In [28]:
# first pass, grouping by 'Plant'
byplant = byunit[[
    'Plant', 'Capacity (MW)', 'Latitude', 'Tracker ID', 'Combustion technology',
    'Start Year', 'Retire Year', 'Annual CO2 (million tonnes / annum)', 'Year',
    'Planned Retire','heatmw','emissionmw'
]].groupby(['Plant', 'Year']).agg({
    'Capacity (MW)': 'sum',
    'Latitude': 'mean',
    'Tracker ID': 'first',
    'Combustion technology': 'first',
    'Start Year': ['min', 'max'],
    'Retire Year': ['min', 'max'],
    'Planned Retire': 'first',
    'Annual CO2 (million tonnes / annum)': 'sum',
    'heatmw': 'sum',
    'emissionmw': 'sum'
}).reset_index()
byplant.columns = byplant.columns.get_level_values(0)

#byplant.sort_values(by=['Plant', 'Year'], inplace=True)
byplant.columns = [
    'Plant', 'Year',  'Capacity (MW)','Latitude','Tracker ID', 'Type','start1', 'start2','retire1',
    'retire2', 'Planned Retire',
      'Annual CO2 (million tonnes / annum)', 'heatmw','emissionmw'
]

#byplant.head()
#pd.options.display.float_format = '{:,.7f}'.format

#byplant.loc[byplant['Plant'].str.contains('Drax')]

#operating.head()

In [29]:
#byplant[byplant.Plant.str.contains('Drax')]

### group the units by latitude

In [30]:
# second pass, grouping by 'Latitude'


# round the latitudes before grouping to iron our random 12dp differences!

byplant['Latitude'] = byplant['Latitude'].round(8)

results = byplant.groupby(['Latitude','Year']).agg(
    {'Capacity (MW)' : 'sum',
     'Plant' : 'first',
     'Type' : ['first','last'],
    'Tracker ID' : 'first',
     'start1' : 'min',
     'start2' : 'max',
     'retire1' : 'min',
     'retire2' : 'max',
     'Planned Retire' : 'first',
    'Annual CO2 (million tonnes / annum)' : 'sum',
     'heatmw':'sum',
     'emissionmw': 'sum'
    }).reset_index()

results.columns = results.columns.get_level_values(0)

results.sort_values(by=['Plant','Year'], inplace=True)

results.columns = ['Latitude','Year','Capacity (MW)','Plant','Type1','Type2','Tracker ID','start1','start2','retire1','retire2','Planned Retire','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']

# combine the non-unique first/last plant types

results['Type'] = np.where(results['Type1'] == results['Type2'],
                             results['Type1'],
                             results['Type1'] + ' / ' + results['Type2'])
                           
# we need to retain Plant1 to use for merging the metadata later on
# but we don't need the latitude any more as we'll grab it again with metadata

results = results[['Plant','Capacity (MW)','Year','Type','Tracker ID','Annual CO2 (million tonnes / annum)','start1','start2','retire1','retire2','Planned Retire','heatmw','emissionmw']]
#results.head()



### find the units operating into the future

In [31]:
# find the units that are still operating with no plans to retire
# find the units that are operating
remaining = df.loc[(df['Status'] == 'Operating')]
# find units that have no entry against 'retire year' or 'planned retire'
remaining = remaining.loc[(remaining['Planned Retire'].isnull()) & (remaining['Retire Year'].isnull())]
# first pass, grouping by 'Plant'
remaining_byplant = remaining[
        ['Plant', 'Capacity (MW)','Latitude','Tracker ID','Combustion technology','Annual CO2 (million tonnes / annum)','Start Year','Retire Year','heatmw','emissionmw']
    ].groupby('Plant').agg(
        {'Capacity (MW)' : 'sum',
         'Latitude' : 'mean',
         'Tracker ID' : 'first',
         'Combustion technology' : 'first',
         'Start Year' : ['min','max'],
         'Retire Year' : ['min','max'], 
         'Annual CO2 (million tonnes / annum)' : 'sum',
         'heatmw': 'sum',
         'emissionmw': 'sum'
        }).reset_index()




# add a year flag
remaining_byplant['Year'] = current_year
# flatten the results
remaining_byplant.columns = remaining_byplant.columns.get_level_values(0)
remaining_byplant.columns = ['Plant','Capacity (MW)','Latitude','Tracker ID','Combustion technology','start1','start2','retire1','retire2','Annual CO2 (million tonnes / annum)','heatmw','emissionmw','Year']




#remaining_byplant.loc[remaining_byplant['Plant'].str.contains('Belchatow')]
#remaining_byplant.head()

In [32]:
# second pass, grouping by 'Latitude'

remaining_byplant['Latitude'] = remaining_byplant['Latitude'].round(8)

remaining_plants = remaining_byplant.groupby(['Latitude']).agg(
    {'Capacity (MW)' : 'sum',
     'Plant' : 'first',
     'Combustion technology' : ['first','last'],
    'Tracker ID' : 'first',
     'Year' : 'first',
     'start1' : 'min',
     'start2' : 'max',
     'retire1' : 'min',
     'retire2' : 'max', 
    'Annual CO2 (million tonnes / annum)' : 'sum',
     'heatmw':'sum',
     'emissionmw':'sum'
    }).reset_index()
remaining_plants.columns = remaining_plants.columns.get_level_values(0)

remaining_plants.columns = ['Latitude','Capacity (MW)','Plant','Type1','Type2','Tracker ID','Year','start1','start2','retire1','retire2','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']
remaining_plants.sort_values(by=['Plant','Year'], inplace=True)


remaining_plants['Type'] = np.where(remaining_plants['Type1'] == remaining_plants['Type2'],
                             remaining_plants['Type1'],
                             remaining_plants['Type1'] + ' / ' + remaining_plants['Type2'])

remaining_plants = remaining_plants[['Plant','Capacity (MW)','Year','Type','Tracker ID','Annual CO2 (million tonnes / annum)','start1','start2','retire1','retire2','heatmw','emissionmw']]

# make sure all plants have a retire year
remaining_plants['retire1'].fillna(2100, inplace=True)
remaining_plants['retire2'].fillna(2100, inplace=True)


#remaining_plants.head()
#remaining_plants.loc[remaining_plants['Plant'].str.contains('Belchatow')]


In [33]:
# append the remaining plants
results = results.append(remaining_plants,sort=False)
results.sort_values(by=['Plant','Year'], inplace=True)


In [34]:
# group by plant and capacity
# ie get one row for each capacity the plant has had, with a range of years for that capacity
grouped = results.groupby(
    ['Plant','Capacity (MW)']).agg(
    {'Year' : ['min','max'],
    'Type' : 'last',
    'Tracker ID' : 'first',
     'start1' : 'min',
     'start2' : 'max',
     'retire1' : 'min',
     'retire2' : 'max',
     'Planned Retire' : 'first',
    'Annual CO2 (million tonnes / annum)' : 'first',
     'heatmw':'first',
     'emissionmw':'first'
    }).reset_index()

# flatten the output, joining with "_"
grouped.columns = grouped.columns.get_level_values(0)

# an alternative way to flatten
# grouped.columns = ["_".join(x) for x in results.columns.ravel()]


# manually rename the columns
grouped.columns = ['Plant', 'Capacity (MW)','year1','year2','Type','Tracker ID','start1','start2','retire1', 'retire2','Planned Retire','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']

# drop the TrackerID, we don't need it
#grouped = grouped[['Plant', 'Capacity (MW)','year1','year2','Annual CO2 (million tonnes / annum)','Type','start1','start2','retire1', 'retire2','Planned Retire']]


grouped['retire3'] = np.where(grouped['Planned Retire'].notnull(),
                           current_year,
                            2100)

# sort the results
grouped.sort_values(by=['Capacity (MW)','Plant'], ascending=False, inplace=True)



# save the grouped results to a csv
# grouped.to_csv('time_series.csv', encoding='utf-8-sig')
#grouped.head()
#grouped[grouped.Plant.str.contains('Drax')]
#grouped[grouped.retire3!=2020]

In [35]:
# get metadata for each plant

metadata = open_closed[
    ['Plant',
          'Country',
          'Coal type',
          'Region',
          'Latitude',
          'Longitude',
    ]].groupby(['Plant']).agg({'Country' : 'first',
                               'Coal type' : ['first', 'last'],
                               'Region' : 'first',
                               'Latitude' : 'mean',
                               'Longitude' : 'mean',
                              }).reset_index()

# flatten the output
metadata.columns = metadata.columns.get_level_values(0)

# manually rename the columns
metadata.columns = ['Plant', 'Country','coal1','coal2', 'Region','Latitude', 'Longitude']

metadata['Coal type'] = np.where(metadata['coal1'] == metadata['coal2'],
                             metadata['coal1'],
                             metadata['coal1'] + ' / ' + metadata['coal2'])

metadata['Status'] = 'Operating'

metadata = metadata[['Plant','Country','Region','Latitude','Longitude','Coal type','Status']]
#metadata.head()

#metadata.loc[metadata['Plant'].str.contains('Belchatow')]
#metadata.loc[metadata['Type1'] != metadata['Type2']]

In [36]:
# create a combined time series with metadata attached
combined = pd.merge(grouped,metadata,left_on=['Plant'],right_on=['Plant'], how='inner')

#combined.head()
#combined.loc[combined['Plant'].str.contains('Drax')]



In [37]:
# calculate a CO2 factor to multiply with the capacity and annual capacity factors

combined['co2fac'] = combined['heatmw']/combined['Capacity (MW)']*combined['emissionmw']/combined['Capacity (MW)']*9.2427/1000000000000

combined.drop({'heatmw','emissionmw'},inplace=True,axis=1)
#combined.head()

In [38]:
# top 10 largest by MW 

combined[['Plant','Capacity (MW)','Type','start1','start2','Country']][(combined['year2'] == 2021)].head(15)

,Plant,Capacity (MW),Type,start1,start2,Country
6,Taean power station,6446.0,igcc,1995.0,2017.0,South Korea
12,Dangjin power station,6040.0,supercritical,1999.0,2016.0,South Korea
18,Taichung power station,5500.0,subcritical,1991.0,2006.0,Taiwan
22,Belchatow power station,5030.0,subcritical,1983.0,2011.0,Poland
30,Yeongheung power station,5080.0,supercritical,2004.0,2014.0,South Korea
44,Medupi Power Station,4768.8,supercritical,2015.0,2021.0,South Africa
65,Neurath power station,4112.0,Unknown,1972.0,2012.0,Germany
78,Duyen Hai Power Generation Complex,4378.0,Unknown,2015.0,2021.0,Vietnam
82,Vinh Tan power station,4244.0,subcritical,2014.0,2019.0,Vietnam
90,Drax power station,1402.0,subcritical,1985.0,1986.0,United Kingdom


In [39]:
#combined[['Plant','Capacity (MW)','Type','year1','year2','start1','start2','Country']][combined.Plant.str.contains('Beilun')]

# work on the pipeline

In [40]:
# select pipeline
pipeline = df.loc[(df['Status'] == 'Announced') | (df['Status'] == 'Construction') | (df['Status'] == 'Pre-permit') | (df['Status'] == 'Permitted')]

# select only the columns we care about
pipeline = pipeline[['Unit',
                     'Status',
                      'Plant',
                      'Country',
                      'Region',
                      'Capacity (MW)',
                      'Combustion technology',
                      'Coal type',
                      'Latitude',
                      'Longitude',
                      'Annual CO2 (million tonnes / annum)',
                     'heatmw',
                     'emissionmw'
                    ]]
#pipeline.head()

In [41]:
# group the pipeline by plant and status

pipeline_plants1 = pipeline[['Plant', 'Capacity (MW)','Status','Latitude','Combustion technology','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']].groupby(
    ['Plant','Status']).agg(
    {'Capacity (MW)' : 'sum',
     'Combustion technology' : ['first'],
     'Annual CO2 (million tonnes / annum)' : 'sum',
     'Latitude' : 'mean',
     'heatmw':'sum',
     'emissionmw':'sum'
    }).reset_index()

# flatten the output, joining with "_"
# grouped.columns = ["_".join(x) for x in results.columns.ravel()]
# an alternative way to flatten
pipeline_plants1.columns = pipeline_plants1.columns.get_level_values(0)

pipeline_plants1.columns = ['Plant','Status','Capacity (MW)','Combustion technology','Annual CO2 (million tonnes / annum)','Latitude','heatmw','emissionmw']

# sort the results
#pipeline_plants.sort_values(by=['Plant','Status'], inplace=True)

# save the grouped results to a csv
# grouped.to_csv('time_series.csv', encoding='utf-8-sig')
#pipeline_plants = pipeline_plants[['Plant','Capacity (MW)','Status']]
#pipeline_plants1.head()

#pipeline_plants1.loc[pipeline_plants1['Plant'].str.contains('Bereket')]


In [42]:
# second pass, grouping by 'Latitude'
pipeline_plants = pipeline_plants1.groupby(['Latitude','Status']).agg(
    {'Capacity (MW)' : 'sum',
     'Plant' : 'first',
     'Combustion technology' : ['first','last'],
    'Annual CO2 (million tonnes / annum)' : 'sum',
     'heatmw':'sum',
     'emissionmw':'sum'
    }).reset_index()

pipeline_plants.columns = pipeline_plants.columns.get_level_values(0)

#results.sort_values(by=['Plant','Year'], inplace=True)

pipeline_plants.columns = ['Latitude','Status','Capacity (MW)','Plant','Type1','Type2','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']

# combine the non-unique first/last plant names

pipeline_plants['Type'] = np.where(pipeline_plants['Type1'] == pipeline_plants['Type2'],
                             pipeline_plants['Type1'],
                             pipeline_plants['Type1'] + ' / ' + pipeline_plants['Type2'])
                           
pipeline_plants['Year'] = current_year -1

pipeline_plants = pipeline_plants[['Plant','Capacity (MW)','Year','Status','Type','Annual CO2 (million tonnes / annum)','heatmw','emissionmw']]
#pipeline_plants.head()
#pipeline_plants.loc[pipeline_plants['Plant'].str.contains('Patuakhali')]

In [43]:
# get metadata for the pipeline

pipe_metadata = pipeline[
    ['Plant',
          'Country',
          'Coal type',
          'Region',
          'Latitude',
          'Longitude',
    ]].groupby(['Plant']).agg({'Country' : 'first',
                                                'Coal type' : 'first',
                                                'Region' : 'first',
                                                'Latitude' : 'mean',
                                                'Longitude' : 'mean',
                                                }).reset_index()

# flatten the output
pipe_metadata.columns = pipe_metadata.columns.get_level_values(0)
# manually rename the columns
#pipe_metadata.columns = ['Plant', 'Type', 'Longitude', 'Latitude', 'Country','Region','Estimated annual CO2 (MtCO2)','start1','start2','coal_type']

#pipe_metadata.head()

In [44]:
# create a combined pipeline with metadata attached
pipe_combined = pd.merge(pipeline_plants,pipe_metadata,left_on=['Plant'],right_on=['Plant'], how='inner')

# add columns so the shape matches the operating plants data
pipe_combined['year1'] = pipe_combined['year2'] = current_year
pipe_combined['retire1'] = pipe_combined['retire2'] = 2100
pipe_combined['start1'] = pipe_combined['start2'] = 2100

# add a random offset to the lat/long
# (separate out announced vs pre-permit vs constuction etc)
# ±0.00025 is roughly ±50m
# https://urldefense.proofpoint.com/v2/url?u=https-3A__en.wikipedia.org_wiki_Decimal-5Fdegrees-255Cn&d=DwIG-g&c=-35OiAkTchMrZOngvJPOeA&r=8-uRO8MVZ_RG3ApGfbZ_IQ&m=wKLMWxgrkaZ5TNX-2_uFqyo8gG3CPKVda6F9uMESr_0qJnMUUCi-biu5OrowMdiH&s=024wMUCnmGuR9kvCYqkoXaExEKLNjv3mfL9a_fscW2o&e= pipe_combined['Latitude'] = pipe_combined['Latitude'] + np.random.uniform(-0.00025,0.00025,pipe_combined['Latitude'].shape)
pipe_combined['Longitude'] = pipe_combined['Longitude'] + np.random.uniform(-0.00025,0.00025,pipe_combined['Longitude'].shape)

#pipe_combined.head()



In [45]:
# add column for a co2 factor to multiply with capacity and capacity factor

pipe_combined['co2fac'] = pipe_combined['heatmw']/pipe_combined['Capacity (MW)']*pipe_combined['emissionmw']/pipe_combined['Capacity (MW)']*9.2427/1000000000000

pipe_combined.drop({'heatmw','emissionmw'},inplace=True,axis=1)
#pipe_combined.head()

In [46]:
# re-order the columns to match the combined operating plants data

pipe_combined = pipe_combined[['Plant',
                               'Capacity (MW)',
                               'year1',
                               'year2',
                               'Annual CO2 (million tonnes / annum)',
                               'Type',
                               'start1',
                               'start2',
                               'retire1',
                               'retire2',
                               'Country',
                               'Region',
                               'Latitude',
                               'Longitude',
                               'Coal type',
                               'Status',
                              'co2fac']]

#pipe_combined.head()

# create the final map datafile

In [47]:
# create final data for the map, including pipeline

coal_map = combined.append(pipe_combined)
coal_map.head()

coal_map['Annual CO2 (million tonnes / annum)'] = coal_map['Annual CO2 (million tonnes / annum)'].round(1)
#coal_map['Country'].replace('United States', 'US', inplace=True)
#coal_map['Country'].replace('United Kingdom', 'UK', inplace=True)



coal_map['Longitude'] = coal_map['Longitude'].round(6)
coal_map['Latitude'] = coal_map['Latitude'].round(6)







In [48]:
coal_map['retire_label1'] = coal_map['retire1']
coal_map['retire_label2'] = coal_map['retire2']


coal_map['start_label1'] = coal_map['start1']
coal_map['start_label2'] = coal_map['start2']



coal_map['retire_label1'].replace(2100, '?', inplace=True)
coal_map['retire_label2'].replace(2100, '?', inplace=True)


coal_map['start_label1'].replace(1900, '?', inplace=True)
coal_map['start_label2'].replace(1900, '?', inplace=True)

coal_map['start_label1'].replace(2100, '?', inplace=True)
coal_map['start_label2'].replace(2100, '?', inplace=True)

#coal_map['start_label1'] = coal_map['start_label1'].round(0)
#coal_map['start_label2'] = coal_map['start_label2'].round(0)


coal_map['start_label1'] = coal_map.start_label1.astype(str)
coal_map['start_label2'] = coal_map.start_label2.astype(str)
coal_map['retire_label1'] = coal_map.retire_label1.astype(str)
coal_map['retire_label2'] = coal_map.retire_label2.astype(str)


coal_map['start_label'] = np.where(coal_map['start_label1'] == coal_map['start_label2'],
                                  coal_map['start_label1'],
                                  coal_map['start_label1'] + ' - ' + coal_map['start_label2'])

coal_map['retire_label'] = np.where(coal_map['retire_label1'] == coal_map['retire_label2'],
                                  coal_map['retire_label1'],
                                 coal_map['retire_label1'] + ' - ' + coal_map['retire_label2'])

coal_map['start_label'] = coal_map['start_label'].str.replace(r'\.0','')
coal_map['retire_label'] = coal_map['retire_label'].str.replace(r'\.0','')
coal_map.drop('start_label1',inplace=True,axis=1)
coal_map.drop('start_label2',inplace=True,axis=1)
coal_map.drop('retire_label1',inplace=True,axis=1)
coal_map.drop('retire_label2',inplace=True,axis=1)

#coal_map.head()

C:\Users\chiga\AppData\Local\Temp/ipykernel_33076/3039271746.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  coal_map['start_label'] = coal_map['start_label'].str.replace(r'\.0','')
C:\Users\chiga\AppData\Local\Temp/ipykernel_33076/3039271746.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  coal_map['retire_label'] = coal_map['retire_label'].str.replace(r'\.0','')


In [49]:
region_label1 = []
for country in coal_map['Country']:
    if (country == 'China'):
        region_label1.append('China')
    elif (country == 'India'):
        region_label1.append('India')
    elif (country == 'United States'):
        region_label1.append('United States')
    elif (country == 'United Kingdom'):
        region_label1.append('EU27+UK')
    else:
        region_label1.append(np.nan)        
coal_map['region_label1'] = region_label1

#coal_map[['region_label1','Country']].head(50)


In [50]:
coal_map['Region'].replace('non-EU Europe','Non-EU Europe',inplace=True)

region_label2 = []
for region in coal_map['Region']:
    if (region == 'EU27'):
        region_label2.append('EU27+UK')
    elif ((region == 'East Asia') | (region == 'South Asia') | (region == 'SE Asia')):
        region_label2.append('Other Asia')
    elif (region == 'Eurasia'):
        region_label2.append('Former USSR')
    elif (region =='Africa and Middle East'):
        region_label2.append('Africa and Middle East')
    elif (region =='Non-EU Europe'):
        region_label2.append('Non-EU Europe')
    elif (region =='Latin America'):
        region_label2.append('Latin America')
    else:
        region_label2.append('Other')        
        
coal_map['region_label2'] = region_label2
#coal_map[['region_label2','Country']].head(50)

In [51]:
#coal_map[coal_map.Country=='Germany'].head()

In [52]:
coal_map['region_label'] = np.where((coal_map['Country'] == 'China') | (coal_map['Country'] == 'India') | (coal_map['Country'] == 'United States') | (coal_map['Country'] == 'United Kingdom'),
                             coal_map['region_label1'],
                             coal_map['region_label2'])
coal_map.drop('region_label1',inplace=True,axis=1)
coal_map.drop('region_label2',inplace=True,axis=1)
coal_map.drop('Region',inplace=True,axis=1)
coal_map.drop('Planned Retire',inplace=True,axis=1)
coal_map.drop('retire_label',inplace=True,axis=1)
coal_map.sort_values(by=['Capacity (MW)'], inplace=True,ascending=False)

#coal_map.head()

In [53]:
# correct country name "Poland " with trailing space

coal_map['Country'] = np.where(coal_map['Country']=='Poland ','Poland',coal_map['Country'])

# Country- and year-specific CO2 calculation

In [54]:
# load Ember global electricity data

name = 'Data-Global-Electricity-Review-2021'

cf = pd.read_excel(name + '.xlsx', sheet_name='Data')

# use first row as column headers

cf.columns = cf.iloc[0]

# drop anything that doesn't relate to coal

cf = cf[cf.Variable=='Coal']

# convert TWh and GW to numeric

cf['Generation (TWh)'] = cf['Generation (TWh)'].apply(pd.to_numeric, errors='coerce')
cf['Capacity (GW)'] = cf['Capacity (GW)'].apply(pd.to_numeric, errors='coerce')
cf['Year'] = cf['Year'].apply(pd.to_numeric, errors='coerce')

# replace NaN capacity with "0"

cf['Capacity (GW)'] = np.where(cf['Capacity (GW)'].isna(),0,cf['Capacity (GW)'])

In [55]:
# calculate per-country-per-year cf

cf['cap_fac'] = cf['Generation (TWh)'].div(cf['Capacity (GW)']*24*365)*1000

# adjust down calculated capacity factors if greater than 95%

cf['cap_fac'] = np.where((cf['cap_fac']>0.95)&(cf['cap_fac']<5),0.95,cf['cap_fac'])

In [56]:
# create pivot table of capacity factors for each country in each year

capfac1 = cf[['Year','Area','cap_fac']].pivot_table(index='Area',columns='Year',values='cap_fac').reset_index()

# rename Area to County

capfac1 = capfac1.rename(columns={'Area':'Country'})

# make column headers into strings not numbers

capfac1.columns = capfac1.columns.map(str)


#capfac1.head()

In [57]:
# use 2019 value where 2020 is missing

capfac1['2020'] = np.where(capfac1['2020'].isna(),capfac1['2019'],capfac1['2020'])
capfac1['2020'] = np.where(capfac1['2020']==np.inf,capfac1['2019'],capfac1['2020'])

# add 2021 entry using 2020 value

capfac1['2021'] = capfac1['2020']

#capfac1.head()

In [58]:
# merge full list of tracker countries into capfac

# create list of unique countries from the coal map data

countries = pd.DataFrame()
countries['Country'] = coal_map.Country.unique()

# merge

capfac = pd.merge(capfac1,countries,left_on=['Country'],right_on=['Country'],how='outer',indicator=True)

#capfac.tail(50)

In [59]:
# double check that the "World" averages are in row 86

capfac.Country.iloc[86]

'World'

In [60]:
# use World average where no entry

# replace inf and zero with nan

for column in capfac.columns:
    capfac[column] = np.where(capfac[column]==np.inf,np.nan,capfac[column])
    capfac[column] = np.where(capfac[column]==0,np.nan,capfac[column])

# loop through all columns one at a time and use the World value from row 86 if NaN

for column in capfac.columns:
    capfac[column].fillna(capfac[column].iloc[86],inplace=True)


    
#capfac.head(50)

In [61]:
# merge capfac data into map file

coal_map = pd.merge(coal_map,capfac,left_on=['Country'],right_on=['Country'], how='left')

#coal_map.columns

In [62]:
# calculate the annual emissions and generation for each year

columns = ['2000','2001','2002','2003','2004','2005','2006',
           '2007','2008','2009','2010','2011','2012','2013',
           '2014','2015','2016','2017','2018','2019','2020','2021']

for column in columns:
    coal_map['co2_'+column] = (coal_map['co2fac']*coal_map[column]*coal_map['Capacity (MW)']).round(1)
    coal_map['TWh_'+column] = (coal_map[column]*coal_map['Capacity (MW)']*365*24/1000000).round(1)
    coal_map.drop({column},inplace=True,axis=1)

# drop the capacity factor estimates to save on weight for the coal map file

coal_map.drop({'Annual CO2 (million tonnes / annum)','co2fac','Tracker ID','_merge'},inplace=True,axis=1)

In [63]:
# show results

coal_map[['Plant','Country','Type','Capacity (MW)','co2_2000','co2_2010','co2_2020','TWh_2000','TWh_2010','TWh_2020']][(coal_map.Status=='Operating')&(coal_map.Country=='United Kingdom')].head()

,Plant,Country,Type,Capacity (MW),co2_2000,co2_2010,co2_2020,TWh_2000,TWh_2010,TWh_2020
33,Drax power station,United Kingdom,subcritical,4206.0,15.8,15.6,3.8,14.6,14.4,3.5
92,Drax power station,United Kingdom,subcritical,3505.0,13.2,13.0,3.2,12.1,12.0,2.9
167,Drax power station,United Kingdom,subcritical,2804.0,10.5,10.4,2.6,9.7,9.6,2.4
275,Longannet power station,United Kingdom,subcritical,2456.0,9.2,9.1,2.2,8.5,8.4,2.1
371,West Burton power station,United Kingdom,subcritical,2188.0,8.2,8.1,2.0,7.6,7.5,1.8


In [64]:
#coal_map.head()

In [65]:

# save the final results to a csv
# index=False to omit the unnecessary index column
coal_map.to_csv('coal_map_2022.csv', encoding='utf-8-sig', index=False)

In [66]:
coal_map.columns

Index(['Plant', 'Capacity (MW)', 'year1', 'year2', 'Type', 'start1', 'start2',
       'retire1', 'retire2', 'retire3', 'Country', 'Latitude', 'Longitude',
       'Coal type', 'Status', 'start_label', 'region_label', 'co2_2000',
       'TWh_2000', 'co2_2001', 'TWh_2001', 'co2_2002', 'TWh_2002', 'co2_2003',
       'TWh_2003', 'co2_2004', 'TWh_2004', 'co2_2005', 'TWh_2005', 'co2_2006',
       'TWh_2006', 'co2_2007', 'TWh_2007', 'co2_2008', 'TWh_2008', 'co2_2009',
       'TWh_2009', 'co2_2010', 'TWh_2010', 'co2_2011', 'TWh_2011', 'co2_2012',
       'TWh_2012', 'co2_2013', 'TWh_2013', 'co2_2014', 'TWh_2014', 'co2_2015',
       'TWh_2015', 'co2_2016', 'TWh_2016', 'co2_2017', 'TWh_2017', 'co2_2018',
       'TWh_2018', 'co2_2019', 'TWh_2019', 'co2_2020', 'TWh_2020', 'co2_2021',
       'TWh_2021'],
      dtype='object')

# Data for the article

In [67]:
# make a copy to use for analysis

df2 = df.copy()

In [68]:
# print summary stats of the data
# use these numbers for the article intro etc
# using copy of orig data (df2) rather than version modified for the map

values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Operating')].sum().round(0)), ' MW operating']
print(*values)
values =[(df2['Capacity (MW)'].loc[(df2['Status']=='Operating')].count()), ' units operating']
print(*values)
print()
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Retired')].sum().round(0)), ' MW retired']
print(*values)
values = [(df2['Capacity (MW)'].loc[((df2['Status']=='Retired')&(df2['Retire Year']>2009))].sum()), ' MW retired since 2010']
print(*values)
values = [((df['Capacity (MW)'].loc[(
    (df['Status'] == 'Operating') |
    (df['Status'] == 'Mothballed'))].sum() - df['Capacity (MW)'].loc[(
        (df['Status'] == 'Operating') |
        (df['Status'] == 'Mothballed')) & (df['Retire Year'].isnull()) & (
            df['Planned Retire'].isnull())].sum())).round(0), ' MW expected to retire'] # need to use df not df2 here as df2 contains dummy retire years
print(*values)
print()
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Construction')].sum()), ' MW being built']
print(*values)
print()
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Announced')|(df2['Status']=='Pre-permit')|(df2['Status']=='Permitted')].sum()), ' MW planned']
print(*values)
print()
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Announced')].sum()), ' MW announced']
print(*values)
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Pre-permit')].sum()), ' MW pre']
print(*values)
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Permitted')].sum()), ' MW permitted']
print(*values)
print()
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Cancelled')].sum()), ' MW cancelled']
print(*values)
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Shelved')].sum()), ' MW shelved']
print(*values)
values = [(df2['Capacity (MW)'].loc[(df2['Status']=='Mothballed')].sum().round(0)), ' MW mothballed']
print(*values)

#df['Status'].dtypes

2067116.0  MW operating
6559  units operating

432509.0  MW retired
363416.99  MW retired since 2010
601036.0  MW expected to retire

178058.0  MW being built

298114.5  MW planned

118832.0  MW announced
100416.0  MW pre
78866.5  MW permitted

1760324.9  MW cancelled
112259.0  MW shelved
24885.0  MW mothballed


## code to spit out pipeline data
* broken down by country and type (construction, announced, etc)
* next cell calculates country shares of the pipeline
* saves pipeline table in CSV "pipeline_20XX.csv"

In [69]:
plans = df2[['Country','Capacity (MW)','Status']][(df2.Status=='Announced')|(df2.Status=='Construction')|(df2.Status=='Permitted')|(df2.Status=='Pre-permit')].groupby(['Country','Status']).agg({'Capacity (MW)':['sum']}).reset_index()
plans.columns = plans.columns.get_level_values(0)
plans = plans.pivot_table(index='Country', columns='Status', values='Capacity (MW)', aggfunc='sum').reset_index()
plans.fillna(0,inplace=True)


plans['Pipeline']=plans['Announced']+plans['Construction']+plans['Permitted']+plans['Pre-permit']

plans
plans.sort_values(by=['Pipeline'],inplace=True,ascending=False)
plans.reset_index(inplace=True)

plans['Status']=plans.index+1
plans

Status,index,Country,Announced,Construction,Permitted,Pre-permit,Pipeline,Status
0,7,China,88190.0,93777.0,50665.0,57922.0,290554.0,1
1,10,India,5320.0,31300.0,13210.0,7213.0,57043.0,2
2,11,Indonesia,600.0,18699.0,2020.0,4640.0,25959.0,3
3,36,Vietnam,0.0,7380.0,2660.0,5590.0,15630.0,4
4,2,Bangladesh,6350.0,6734.0,0.0,2020.0,15104.0,5
5,32,Turkey,3188.0,1520.0,3420.0,4100.0,12228.0,6
6,20,Mongolia,1350.0,50.0,450.0,5980.0,7830.0,7
7,17,Laos,4786.0,0.0,2000.0,350.0,7136.0,8
8,38,Zimbabwe,3300.0,940.0,1120.0,750.0,6110.0,9
9,23,Pakistan,40.0,2640.0,361.5,2720.0,5761.5,10


In [70]:
pd.options.display.float_format = '{:,.0f}'.format
plans_share = plans[['Country','Announced','Construction','Permitted','Pre-permit','Pipeline']].iloc[:,1:].apply(lambda x: x / float(x.sum())*100, axis=0).reset_index()
plans_share.rename(columns={'index':'Status','Announced':'Announced_%','Construction':'Construction_%','Permitted':'Permitted_%','Pre-permit':'Pre-permit_%','Pipeline':'Pipeline_%'},inplace=True)
plans_share['Status'] = plans_share['Status']+1
plans_share
plans = pd.merge(plans,plans_share,left_on=['Status'],right_on=['Status'], how='inner')
plans.to_csv('Pipeline_'+str(current_year)+'.csv',index=False)
plans

Status,index,Country,Announced,Construction,Permitted,Pre-permit,Pipeline,Status,Announced_%,Construction_%,Permitted_%,Pre-permit_%,Pipeline_%
0,7,China,"88,190","93,777","50,665","57,922","290,554",1,74,53,64,58,61
1,10,India,"5,320","31,300","13,210","7,213","57,043",2,4,18,17,7,12
2,11,Indonesia,600,"18,699","2,020","4,640","25,959",3,1,11,3,5,5
3,36,Vietnam,0,"7,380","2,660","5,590","15,630",4,0,4,3,6,3
4,2,Bangladesh,"6,350","6,734",0,"2,020","15,104",5,5,4,0,2,3
5,32,Turkey,"3,188","1,520","3,420","4,100","12,228",6,3,1,4,4,3
6,20,Mongolia,"1,350",50,450,"5,980","7,830",7,1,0,1,6,2
7,17,Laos,"4,786",0,"2,000",350,"7,136",8,4,0,3,0,1
8,38,Zimbabwe,"3,300",940,"1,120",750,"6,110",9,3,1,1,1,1
9,23,Pakistan,40,"2,640",362,"2,720","5,762",10,0,1,0,3,1


# make the map linechart (capacity by country and year)
* send the results to 'line_chart.csv'
* complete breakdown to 'country_by_MW_by_yr.csv'

In [71]:
# create a temp df to use for the analysis

temp = df2.copy()
temp['Start Year'].fillna(1900, inplace=True)
temp['Retire Year'].fillna(2100, inplace=True)


In [72]:
# calculate operating capcity in each year, by country

capacity = pd.DataFrame()
for year in range(2000,current_year):
    operating = temp.loc[(temp['Start Year'] <= year) & (temp['Retire Year'] > year) & ((temp.Status=='Operating')|(temp.Status=='Retired'))]
    operating_mw = operating[
        ['Country','Region', 'Capacity (MW)']
    ].groupby('Country').agg(
        {'Capacity (MW)' : 'sum',
         'Region' : 'first'
        }).reset_index()
    operating_mw.columns = operating_mw.columns.get_level_values(0)
    
    operating_mw['year'] = year
    capacity = capacity.append(operating_mw)
capacity.sort_values(by=['Country','year'], inplace=True)
#capacity.head()

In [73]:
# calculate operating capcity in each country, by year

yr_mw = capacity.pivot_table(index='year', columns='Country', values='Capacity (MW)', aggfunc='sum').reset_index()


# calculate a world total
yr_mw['World'] = yr_mw[yr_mw.columns].sum(axis=1)


# get data for the regions

yr_mw_regions = capacity.pivot_table(index='year', columns='Region', values='Capacity (MW)', aggfunc='sum').reset_index()

# combine countries and regions

combined = pd.merge(yr_mw,yr_mw_regions,left_on=['year'],right_on=['year'], how='inner')
combined.head()
combined.to_csv('country_by_MW_by_yr.csv',index=False)
#combined.columns.tolist()
#combined.head()

In [74]:
# define new regions

combined['EU27+UK'] = combined['EU27'] + combined['United Kingdom']
combined['Asia'] = combined['East Asia'] + combined['SE Asia'] + combined['South Asia']
combined['Other Asia'] = combined['Asia'] - combined['China'] - combined['India']
combined['Former USSR'] = combined['Eurasia'] + combined['Ukraine'] + combined['Moldova']
combined['Non-EU Europe'] = combined['non-EU Europe']- combined['Ukraine']- combined['Moldova'] - combined['United Kingdom']
combined['Other'] = combined['World'] - combined['China'] - combined['India'] - combined['Other Asia'] - combined['EU27+UK'] - combined['United States'] - combined['Eurasia'] - combined['Latin America'] - combined['Africa and Middle East'] - combined['non-EU Europe']

# get a table for the map inset chart
pd.options.display.float_format = '{:,.0f}'.format
chart = combined[['year','China','United States','India','Other Asia', 'EU27+UK','Former USSR','Non-EU Europe','Latin America','Africa and Middle East','Other']]

chart.to_csv('line_chart.csv',index=False)
chart

,year,China,United States,India,Other Asia,EU27+UK,Former USSR,Non-EU Europe,Latin America,Africa and Middle East,Other
0,2000,"199,760","334,802","63,214","86,334","190,400","75,807","14,906","10,117","42,804","16,634"
1,2001,"213,083","335,846","64,219","88,958","189,507","76,022","14,906","10,117","44,418","17,246"
2,2002,"223,776","336,247","65,259","97,485","189,081","76,062","14,906","10,117","44,418","18,291"
3,2003,"246,167","334,824","66,844","101,023","187,003","76,294","16,626","10,117","44,418","21,252"
4,2004,"275,634","334,840","67,326","105,611","186,491","76,504","16,626","10,117","44,418","21,253"
5,2005,"322,280","334,990","70,651","106,911","185,601","76,924","17,121","10,117","44,453","19,062"
6,2006,"403,950","335,293","72,939","112,102","184,250","77,224","18,136","10,117","44,453","19,063"
7,2007,"475,765","333,992","77,893","116,785","183,364","77,344","18,214","10,117","44,543","19,880"
8,2008,"529,342","334,730","83,089","119,092","182,612","77,344","18,214","10,117","44,543","19,881"
9,2009,"580,130","335,775","88,550","124,576","183,248","77,764","18,586","10,345","44,543","20,348"


## calculate unit/MW additions and retirements
* overall plus breakdown for China, India, US, EU
* use this for graph showing net change in MW and units
* send results to csv 'net_change.csv'

In [75]:
newunits = pd.DataFrame()
for year in range(2000,current_year):
    new = df2.loc[(df2['Start Year'] == year)&((df2['Status']=='Operating')|(df2['Status']=='Retired')|(df2['Status']=='Mothballed'))]
    #old = df2.loc[(df2['Start Year'] <= year) & (open_closed['Retire Year'] == year)]
    new['year']=year
    #old['year']=year
    newunits = newunits.append(new)
    #units = units.append(old)
newunits = newunits[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()

oldunits = pd.DataFrame()
for year in range(2000,current_year):
    #new = df2.loc[(df2['Start Year'] == year) & (open_closed['Retire Year'] > year)]
    old = df2.loc[(df2['Retire Year'] == year)&(df2['Status']=='Retired')]
    #new['year']=year
    old['year']=year
    oldunits = oldunits.append(old)
    #units = units.append(old)
oldunits = oldunits[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()




newoldunits = pd.merge(newunits,oldunits,left_on=['year'],right_on=['year'], how='outer')


newoldunits.columns = newoldunits.columns.get_level_values(0)


newchina = pd.DataFrame()
for year in range(2000,current_year):
    new = df2.loc[(df2['Country'] == 'China')&(df2['Start Year'] == year)&((df2['Status']=='Operating')|(df2['Status']=='Retired')|(df2['Status']=='Mothballed'))]
    #old = df2.loc[(df2['Start Year'] <= year) & (open_closed['Retire Year'] == year)]
    new['year']=year
    #old['year']=year
    newchina = newchina.append(new)
    #units = units.append(old)
newchina = newchina[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()
newchina.columns = newchina.columns.get_level_values(0)

newindia = pd.DataFrame()
for year in range(2000,current_year):
    new = df2.loc[(df2['Country'] == 'India')&(df2['Start Year'] == year)&((df2['Status']=='Operating')|(df2['Status']=='Retired')|(df2['Status']=='Mothballed'))]
    #old = df2.loc[(df2['Start Year'] <= year) & (open_closed['Retire Year'] == year)]
    new['year']=year
    #old['year']=year
    newindia = newindia.append(new)
    #units = units.append(old)
newindia = newindia[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()
newindia.columns = newindia.columns.get_level_values(0)

oldus = pd.DataFrame()
for year in range(2000,current_year):
    #new = df2.loc[(df2['Start Year'] == year) & (open_closed['Retire Year'] > year)]
    old = df2.loc[(df2['Retire Year'] == year)&(df2['Status']=='Retired')&(df2['Country']=='United States')]
    #new['year']=year
    old['year']=year
    oldus = oldus.append(old)
    #units = units.append(old)
oldus = oldus[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()
oldus.columns = oldus.columns.get_level_values(0)


oldeu = pd.DataFrame()
for year in range(2000,current_year):
    #new = df2.loc[(df2['Start Year'] == year) & (open_closed['Retire Year'] > year)]
    old = df2.loc[(df2['Retire Year'] == year)&(df2['Status']=='Retired')&((df2['Region']=='EU27')|(df2['Country']=='United Kingdom'))]
    #new['year']=year
    old['year']=year
    oldeu = oldeu.append(old)
    #units = units.append(old)
oldeu = oldeu[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()
oldeu.columns = oldeu.columns.get_level_values(0)

oldchina = pd.DataFrame()
for year in range(2000,current_year):
    #new = df2.loc[(df2['Start Year'] == year) & (open_closed['Retire Year'] > year)]
    old = df2.loc[(df2['Retire Year'] == year)&(df2['Status']=='Retired')&(df2['Country']=='China')]
    #new['year']=year
    old['year']=year
    oldchina = oldchina.append(old)
    #units = units.append(old)
oldchina = oldchina[['Capacity (MW)','year']].groupby('year').agg({'Capacity (MW)':['sum','count']}).reset_index()
oldchina.columns = oldchina.columns.get_level_values(0)


newoldunits = pd.merge(newoldunits,newchina,left_on=['year'],right_on=['year'], how='left')
newoldunits = pd.merge(newoldunits,newindia,left_on=['year'],right_on=['year'], how='left')
newoldunits = pd.merge(newoldunits,oldus,left_on=['year'],right_on=['year'], how='left')
newoldunits = pd.merge(newoldunits,oldeu,left_on=['year'],right_on=['year'], how='left')
newoldunits = pd.merge(newoldunits,oldchina,left_on=['year'],right_on=['year'], how='left')

newoldunits.columns = ['year','new MW','new units','retire MW','retire units','new China MW','new China','new India MW','new India','retire US MW','retire US','retire EU MW','retire EU','retire China MW','retire China']
newoldunits.fillna(0,inplace=True)

newoldunits['net GW'] = (newoldunits['new MW'] - newoldunits['retire MW'])/1000.
newoldunits['net units'] = newoldunits['new units'] - newoldunits['retire units']

newoldunits['other new MW'] = newoldunits['new MW'] - newoldunits['new China MW'] - newoldunits['new India MW']
newoldunits['other retire MW'] = newoldunits['retire MW'] - newoldunits['retire US MW'] - newoldunits['retire EU MW'] - newoldunits['retire China MW']
newoldunits['other retire GW'] = - newoldunits['other retire MW']/1000.
newoldunits['retire US GW'] = - newoldunits['retire US MW']/1000.
newoldunits['retire EU GW'] = - newoldunits['retire EU MW']/1000.
newoldunits['retire China GW'] = - newoldunits['retire China MW']/1000.
newoldunits['new China GW'] = newoldunits['new China MW']/1000.
newoldunits['new India GW'] = newoldunits['new India MW']/1000.
newoldunits['other new GW'] = newoldunits['other new MW']/1000.


newoldunits['other new'] = newoldunits['new units'] - newoldunits['new China'] - newoldunits['new India']
newoldunits['other retire'] = newoldunits['retire units'] - newoldunits['retire US'] - newoldunits['retire EU'] - newoldunits['retire China']
newoldunits['other retire'] = - newoldunits['other retire']
newoldunits['retire US'] = - newoldunits['retire US']
newoldunits['retire EU'] = - newoldunits['retire EU']
newoldunits['retire China'] = - newoldunits['retire China']


newoldunits = newoldunits[['year',
                           'new China GW','new India GW','other new GW',
                           'retire US GW','retire EU GW','retire China GW','other retire GW',
                           'net GW',
                           'year',
                           'new China','new India','other new',
                           'retire US','retire EU','retire China','other retire',
                           'net units']]

newoldunits.columns = ['year',
                           'China','India','Other',
                           'US','EU+UK','China','Other',
                           'Net change',
                       'year',
                           'China','India','Other',
                           'US','EU','China','Other',
                           'Net change']

newoldunits.to_csv('net_change.csv',index=False)
#newoldunits.sort_values(by='Other1')
newoldunits



C:\Users\chiga\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\chiga\AppData\Local\Temp/ipykernel_33076/486090872.py:88: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Capacity (MW)_x'} in the result is deprecated and will raise a MergeError in a future version.
  newoldunits = pd.merge(newoldunits,newindia,left_on=['year'],right_on=['year'], how='left')
C:\Users\chiga\AppData\Local\Temp/ipykernel_33076/486090872.py:90: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Capacity (MW)_x'} in the result is deprecated and will raise a MergeError in a future version.
  newoldunits = pd.merge(newoldunits,oldeu,left_on=['year'],right_on=['year'], how='left')


,year,China,India,Other,US,EU+UK,China,Other,Net change,year,China,India,Other,US,EU,China,Other,Net change
0,2000,18,2,15,-0,-3,-0,-1,31,2000,70,9,47,-1,-23,-3,-10,89
1,2001,13,1,7,-0,-1,-0,-1,20,2001,61,6,17,-1,-7,-2,-3,71
2,2002,11,2,11,-1,-2,-0,-1,20,2002,54,7,26,-14,-15,-7,-10,41
3,2003,23,2,7,-2,-3,-0,-0,27,2003,127,7,23,-10,-12,-0,-5,130
4,2004,30,0,6,-1,-1,-0,-1,35,2004,141,3,24,-7,-7,-6,-5,143
5,2005,47,3,4,-0,-1,-0,-3,50,2005,196,15,15,-2,-10,-1,-12,201
6,2006,82,2,7,-0,-1,-1,-0,89,2006,262,17,27,-4,-9,-13,-1,279
7,2007,82,5,8,-3,-1,-10,-1,81,2007,222,27,27,-13,-6,-125,-11,121
8,2008,64,5,4,-1,-1,-10,-0,61,2008,200,29,18,-11,-13,-100,-1,122
9,2009,61,6,11,-2,-1,-11,-0,65,2009,159,28,36,-21,-7,-100,-3,92


# how many countries have coal plants

In [76]:
# how many countries have coal plants

values =[(df2['Country'].loc[(df2['Status']=='Operating')].nunique()), ' operating']
print(*values)

values =[(df2['Country'].loc[(df2['Status']=='Operating')|(df2['Status']=='Construction')|(df2['Status']=='Permitted')|(df2['Status']=='Pre-permit')|(df2['Status']=='Announced')|((df2['Retire Year']>=2000)&(df2['Status']=='Retired'))].nunique()), ' retired since 2000 or operating or planned']
print(*values)

values =[(df2['Country'].loc[(df2['Status']=='Operating')|((df2['Retire Year']>=2000)&(df2['Status']=='Retired'))].nunique()), ' retired since 2000 or operating']
print(*values)

values =[(df2['Country'].loc[(df2['Status']=='Construction')|(df2['Status']=='Permitted')|(df['Status']=='Pre-permit')|(df2['Status']=='Announced')].nunique()), ' planned']
print(*values)

values =[(df2['Country'].loc[(df2['Status']=='Construction')].nunique()), ' being built']
print(*values)

78  operating
90  retired since 2000 or operating or planned
83  retired since 2000 or operating
39  planned
19  being built


In [77]:
values = [(capacity['Country'].loc[capacity['year']==2000].nunique()), ' countries operating coal in 2000']
print(*values)
values =[(capacity['Country'].loc[capacity['year']==2020].nunique()), ' countries operating coal in 2020']
print(*values)
#capacity.loc[(capacity['year']==2017)&(capacity['Capacity (MW)']==0)]

68  countries operating coal in 2000
80  countries operating coal in 2020


## get numbers for the "future" map

In [78]:
still = df[['Country','Region','Capacity (MW)']].loc[(df['Status']=='Operating')&(df['Retire Year'].isnull())&(df['Planned Retire'].isnull())]
still1 = still.groupby(['Region']).sum().reset_index()
still2 = still.groupby(['Country']).sum().reset_index()
still2.columns = ['Region','Capacity (MW)']
still3 = still1.append(still2)
still3.columns = ['Region','Still operating (MW)']
still3

built = df[['Country','Region','Capacity (MW)']].loc[(df['Status']=='Construction')]
built1 = built.groupby(['Region']).sum().reset_index()
built2 = built.groupby(['Country']).sum().reset_index()
built2.columns = ['Region','Capacity (MW)']
built3 = built1.append(built2)
built3.columns = ['Region','Being built (MW)']
built3

planned = df[['Country','Region','Capacity (MW)']].loc[(df['Status']=='Announced')|(df['Status']=='Pre-permit')|(df['Status']=='Permitted')]
planned1 = planned.groupby(['Region']).sum().reset_index()
planned2 = planned.groupby(['Country']).sum().reset_index()
planned2.columns = ['Region','Capacity (MW)']
planned3 = planned1.append(planned2)
planned3.columns = ['Region','Planned (MW)']
planned3

future1 = pd.merge(still3,built3,left_on=['Region'],right_on=['Region'], how='left')
future = pd.merge(future1,planned3,left_on=['Region'],right_on=['Region'], how='left')
future.fillna(0,inplace=True)
future.to_csv('future.csv',index=False)
future.head()

,Region,Still operating (MW),Being built (MW),Planned (MW)
0,Africa and Middle East,"3,197","3,190","10,894"
1,Australia/NZ,"3,783",0,"1,000"
2,East Asia,"1,127,064","102,827","205,057"
3,Eurasia,"36,616",0,"3,029"
4,Latin America,"7,365",240,"1,666"
